In [65]:
from data_utils import *

import numpy as np

import pandas

from matplotlib import pyplot as plt

import pickle

from scipy.interpolate import interp1d

import models

import data_utils

import kalman

from pyro.dynamic import statespace

from datetime import datetime as dt

plt.style.use("ggplot")

datadir = Path(
    r"C:\Users\TDewitte\Desktop\PythonPrograms\S4S\camso-s4s\tdms_data"
)

# Directory where the .pickle files containing processed data will be created
output_dir = Path("./concat_data")

## A new approach with a fresh insight. 
Did we forget to take something into consideration?

First we load the limited amount of data that we will consider.

In [605]:
"""
data_run1_raw = data_utils.load_run(datadir, 1)
data_run1_clean = data_utils.clean_data(data_run1_raw, "out")

data_run2_raw = data_utils.load_run(datadir, 2)
data_run2_clean = data_utils.clean_data(data_run2_raw, "out")

data_run3_raw = data_utils.load_run(datadir, 3)
data_run3_clean = data_utils.clean_data(data_run3_raw, "out")

data_run4_raw = data_utils.load_run(datadir, 4)
data_run4_clean = data_utils.clean_data(data_run4_raw, "out")

data_run5_raw = data_utils.load_run(datadir, 5)
data_run5_clean = data_utils.clean_data(data_run5_raw, "out")

data_run6_raw = data_utils.load_run(datadir, 6)
data_run6_clean = data_utils.clean_data(data_run6_raw, "out")

cdata_all = data_utils.concat_data([data_run1_clean, data_run2_clean, data_run3_clean, data_run4_clean, data_run5_clean, data_run6_clean])
with open(output_dir / "runs_1-5.pickle", "wb") as f:
    pickle.dump(cdata_all, f)
"""

with open("./concat_data/runs_1-5.pickle", "rb") as f:
    cdata_all = pickle.load(f)

with open("./concat_data/runs_11_12_13_14.pickle", "rb") as f:
    cdata_all_2 = pickle.load(f)

## Visualize the data

The data that matters to us at the moment is:
- Vehicle speed
- The load on the midroller
- TMS temperature (considered as ambient)
- Probe temperatures

In [67]:
# print(cdata_all)

## Probe data corrections

The probes for FR in and FR out where swapped for one measurement. We need to correct it in the dataframes.

In [68]:
def swap_probes_between(rdata, date_string_begin, date_string_end):
    # format the dates correctly
    date1=(dt.strptime(date_string_begin, '%Y-%m-%d %H:%M:%S') - pandas.Timestamp("1970-01-01")) // pandas.Timedelta('1s')
    date2=(dt.strptime(date_string_end, '%Y-%m-%d %H:%M:%S') - pandas.Timestamp("1970-01-01")) // pandas.Timedelta('1s')

    # Additional helping column
    rdata['time_column']=cdata_all.probe.index
    rdata['time_column']=pandas.to_datetime(cdata_all.probe['time_column']).astype('int64') / 10**9
    
    df2=rdata.loc[(rdata['time_column'] >= date1/1000) & (rdata['time_column'] <= date2/1000)]
    df2=df2.rename(columns={"probe_FR_IN": "probe_FR_OUT", "probe_FR_OUT": "probe_FR_IN"}) # swap the two columns
    
    # merge them back in the original one
    rdata.update(df2)
    rdata.drop(['time_column'], axis=1)
    return rdata


In [69]:
cdata_all.probe = swap_probes_between(cdata_all.probe, '2022-08-25 17:10:00', '2022-08-25 17:26:00')

In [70]:
def eliminate_probes_between(rdata, date_string_begin, date_string_end, corner, inout):
    # format the dates correctly
    date1=(dt.strptime(date_string_begin, '%Y-%m-%d %H:%M:%S') - pandas.Timestamp("1970-01-01")) // pandas.Timedelta('1s')
    date2=(dt.strptime(date_string_end, '%Y-%m-%d %H:%M:%S') - pandas.Timestamp("1970-01-01")) // pandas.Timedelta('1s')

    # Additional helping column
    rdata['time_column']=cdata_all.probe.index
    rdata['time_column']=pandas.to_datetime(cdata_all.probe['time_column']).astype('int64') / 10**9

    rdata.loc[(rdata['time_column'] >= date1/1000) & (rdata['time_column'] <= date2/1000)] =np.nan

    rdata.drop(['time_column'], axis=1)
    return rdata

In [71]:
cdata_all.probe = eliminate_probes_between(cdata_all.probe, '2022-08-25 14:31:00', '2022-08-25 14:46:00', "RR", "OUT")
cdata_all.probe = eliminate_probes_between(cdata_all.probe, '2022-08-25 17:05:00', '2022-08-25 17:20:00', "FR", "IN")

### Interpolation function of the TMS temperatures

Add the data to the dataframe of the tms.

In [699]:
def interp_TMS(rdata, corner, inout, t0):
    tms_meas = rdata.get_tms_channels(corner=corner, inout=inout, mean=3).dropna()
    
    difference=tms_meas.diff(-5)  # The difference with 20 rows further is just too big.
    tms_down = tms_meas[np.abs(difference)<0.1]

    tms_interp = interp1d(
            get_dt(tms_down, t0), tms_down.values, fill_value="extrapolate", kind='linear'
        )

    return tms_interp  # Returns a function!

def new_tms(rdata, func, corner, inout):
    time_in_s = get_dt(rdata.can.NavSpeed, None)  # Get the time in seconds (and not in this weird date format), use the speed timestamps, as these are continuous
    t_tms_meas = rdata.get_tms_channels(corner=corner, inout=inout, mean=3).dropna().index[0]
    t_gps = rdata.can.NavSpeed.index[0]
    t_gap = (t_tms_meas-t_gps)/ np.timedelta64(1, "s") # If the TMS measurements do not start at the same time as the gps, we need to correct the signal with the time difference
    time_in_s = time_in_s - t_gap
    int_tms = func(time_in_s)

    return int_tms


In [700]:
corner=["RR","RR",'FR','FR']
inout=["IN","OUT","IN","OUT"]

for cornerindex in range(len(corner)):
    interp_tms = interp_TMS(cdata_all, corner[cornerindex], inout[cornerindex],None)
    new_values = new_tms(cdata_all,interp_tms, corner[cornerindex], inout[cornerindex])
    tms_col = f"tms_int_{corner[cornerindex].upper()}_{inout[cornerindex].lower()}"

    cdata_all.can[tms_col]=new_values
    #print(cdata_all.can[tms_col])

for cornerindex in range(len(corner)):
    interp_tms = interp_TMS(cdata_all_2, corner[cornerindex], inout[cornerindex],None)
    new_values = new_tms(cdata_all_2,interp_tms, corner[cornerindex], inout[cornerindex])
    tms_col = f"tms_int_{corner[cornerindex].upper()}_{inout[cornerindex].lower()}"

    cdata_all_2.can[tms_col]=new_values
    #print(cdata_all_2.can[tms_col])
   

## Plot functions

In [701]:
def get_t2_data(rdata, src, corner, inout=None):
    if src == "smarttrack":
        t2 = rdata.smart_track[corner.upper()].dropna()
    elif src == "tms":
        if inout is None:
            raise ValueError("inout required for TMS sensors")
        t2 = rdata.get_tms_channels(corner=corner, inout=inout, mean=3).dropna()  # Adjust if it is the mean, median or minimum
    else:
        raise ValueError(f"Invalid t2 src {src}")
    return t2

In [702]:
def set_axis_dt(ser, t0):
    return ser.set_axis(get_dt(ser, t0))

def plot_data(cdata: data_utils.RunData, T2src, corner, inout):
    # Check if 'corner' and 'length' lists are the same dimensions
    # ...

    t0 = cdata.smart_track.index[0]
   
    fig, axes = plt.subplots(4, 1, figsize=[11, 15], sharex=True)

    # TMS temperature measurement
    for cornerindex in range(len(corner)):
        c = corner[cornerindex]
        i = inout[cornerindex]
        T2_meas = cdata.get_tms_channels(corner=c, inout=i, mean=3).dropna()
        axes[0].plot(
            data_utils.set_axis_dt(T2_meas, t0),
            ls="none",
            marker=".",
            # color="C1",
            label="$T_2$ meas. (TMS)"
        )
    axes[0].set(ylabel="TMS (°C)")

    # TMS temperature interpolation
    for cornerindex in range(len(corner)):
        c = corner[cornerindex]
        i = inout[cornerindex]
        T2_int = cdata.can["tms_int_" +corner[cornerindex].upper()+ "_" + inout[cornerindex].lower()]
    
        axes[0].plot(
            data_utils.set_axis_dt(T2_int, t0),
            # ls="none",
            # marker=".",
            # color="C1",
            label="$T_2$ int. (TMS)"
        )
    axes[0].set(ylabel="TMS (°C)")

    # Probe temperature measurement
    tc = axes[1]#.twinx()
    for cornerindex in range(len(corner)):
        c = corner[cornerindex]
        i = inout[cornerindex]
        tc.plot(
            data_utils.set_axis_dt(cdata.get_probe_in(corner=c, inout=i), t0),
            "-",
            # color="C2",
            label="$T_1$ meas. (probe)"
        )
    
    tc.set(ylabel="Temperature probe (°C)")
    # axes[0].legend()

    # Load
    ta = axes[2]#.twinx()
    for cornerindex in range(len(corner)):
        c = corner[cornerindex]
        i = inout[cornerindex]
        fa_df = data_utils.set_axis_dt(cdata.get_malt_channel(corner=c, inout=i), t0)
        #fa_straddle = data_utils.mask_straddle(fa_df, cdata)
        #ta.plot(fa_straddle, color="C4", lw=6, label="Straddle")
    
        ta.plot(
            fa_df, 
            #color="C1", 
            label=c+i)
    ta.set(ylabel="Load (lbf)")
    ta.grid(False)

    ta.legend(loc=(0.8, 0.75))

    # Speed
    tb = axes[3]#.twinx()
    tb.plot(data_utils.get_dt(cdata.can, t0), cdata.can.NavSpeed, label="Tractor speed")
    tb.set(ylabel="Speed (km/h)", xlabel="Time (s)")
    tb.legend(loc=(0.80, 0.9))

    
    return (fig, axes)

In [703]:
fig1, axes1 = plot_data(cdata_all, corner=["RR","RR",'FR','FR'], inout=["IN","OUT","IN","OUT"], T2src="TMS")

plt.show()

In [704]:
fig2, axes2 = plot_data(cdata_all_2, corner=["RR","RR",'FR','FR'], inout=["IN","OUT","IN","OUT"], T2src="TMS")

plt.show()

In [316]:
# Save again after the small adjustments

with open(output_dir / "runs_1-5.pickle", "wb") as f:
    pickle.dump(cdata_all, f)

In [705]:
import pandas as pd

def collect_runs(data, corner, inout):
    tms_col = f"tms_int_{corner.upper()}_{inout.lower()}"
    run_data = pd.DataFrame(
        {
            "speed": data.can.NavSpeed,
            "load": data.get_malt_channel(corner=corner, inout=inout),
            "tms": data.can[tms_col],
            "probe": data.get_probe_in(corner=corner, inout=inout),
        }
    )
    return run_data

corner = ["RR", "RR", "FR", "FR"]
inout = ["in", "out", "in", "out"]

run_data = {}

for cornerindex in range(len(corner)):
    tms_col = f"{corner[cornerindex].upper()}_{inout[cornerindex].lower()}"
    run_data[tms_col] = collect_runs(cdata_all, corner[cornerindex], inout[cornerindex])

print(run_data)

run_data_2 = {}

for cornerindex in range(len(corner)):
    tms_col = f"{corner[cornerindex].upper()}_{inout[cornerindex].lower()}"
    run_data_2[tms_col] = collect_runs(cdata_all_2, corner[cornerindex], inout[cornerindex])

print(run_data_2)


{'RR_in':                                      speed         load   tms     probe
2022-08-25 14:22:33+00:00         0.011719  6011.543079  11.0       NaN
2022-08-25 14:22:33.100000+00:00  0.011719  6012.242419  11.0       NaN
2022-08-25 14:22:33.200000+00:00  0.011719  6012.943284  11.0       NaN
2022-08-25 14:22:33.300000+00:00  0.011719  6013.645661  11.0       NaN
2022-08-25 14:22:33.400000+00:00  0.011719  6014.349540  11.0       NaN
...                                    ...          ...   ...       ...
2022-08-25 19:31:09.700000+00:00  0.015625  7001.860461  35.0       NaN
2022-08-25 19:31:09.800000+00:00  0.015625  7001.860448  35.0       NaN
2022-08-25 19:31:09.900000+00:00  0.003906  7001.860437  35.0       NaN
2022-08-25 19:31:10+00:00         0.003906  7001.860427  35.0  97.55603
2022-08-25 19:31:10.100000+00:00  0.027344  7001.860419  35.0       NaN

[169924 rows x 4 columns], 'RR_out':                                      speed        load        tms      probe
2022-08-25 

In [711]:
%matplotlib tk

param= [
    #1.128935672263065015e-01,
    #9.890534505140590227e+01,
    #-1.127535937299006807e+00,
    #5.616513408337189617e+01,
    #1.359674042527585414e+01,
    #5.876929627364507525e+01,
    #-4.108624864568822943e+01
    #]

    #6.281302741824414948e-01,  # Best so far
    #8.192221561864243995e+01,
    #6.774627313128092947e-11,
    #4.980182730612345040e+03,
    #1.023906887823085081e-11,
    #2.008025648319744505e-02,
    #1.999297434250133619e-11,

    #3.585276449133369958e-01,
    #1.451191533929677462e+02,
    #-1.754476179819759675e-11,
    #6.691276647160561879e+03,
    #-2.500799912557059020e-11,
    #1.651289931153243629e-02,
    #2.899614723340504744e-12,

    #3.945238108922611842e-01,
    #1.339439003272746049e+02,
    #-3.488400734939600931e-12,
    #9.887526983037561877e+03,
    #6.681230411201682304e-11,
    #2.003328225768336024e-02,
    #3.744467427296629232e-11,

    #4.028013984684544191e-01, # with d0,d1,c0,c1,r0,T0
    #6.436711993957760569e+01,
    #7.490575336229939608e-11,
    #4.667871235944967339e+03,
    #-4.921344004085121844e-11,
    #4.081458862542902688e-02,
    #2.394882160126260828e+01,

    #1.218824974246028203e+00,
    #2.512176951071865005e-03,
    #-1.111671038853071506e-05,
    #5.730562575040962656e+02,
    #5.695592381338392476e-03,
    #2.476648788651818123e+01,

    #1.301533483968141791e+00,
    #5.923694356198438982e-03,
    #-2.937025671664712803e-05,
    #3.185787058706023345e+02,
    #3.520935261213968201e-03,
    #2.469174573451681098e+01,

    #4.718253343712516257e-01,
    #1.642494187758813950e-02,
    #-8.212418405048222693e-05,
    #2.309602786618177106e+02,
    #0,
    #2.493025946458489628e+01,

    1, #8.768923491519737379e-01,  # alpha 5.768923491519737379e-01
    0.006, #1.213441912498325337e-02,  # d0
    -2.5e-05,#-6.023265008693063228e-05,  # d1
    800, #2.650774409078691747e+02,  # r0 --> set
    1200, #8.099895572840328309e-03,  # a_1
    1.1,  # exp #2.498293890371301984e+01,  # T0 --> set, no big impact at the end
    ]
def plot_run(param, cdata):

    run_down = cdata
    alpha = param[0]
    d0= param[1]
    d1= param[2]
    #c0= param[3]
    #c1= param[4]
    r0= param[3]
    a_1= param[4]
    exp=param[5]

    v = run_down['speed'].fillna(method='ffill').to_numpy()
    F = run_down['load'].fillna(method='ffill').to_numpy()
    T = run_down['tms'].fillna(method='ffill').to_numpy()
    P = run_down['probe'].to_numpy()
    
    T1 = T*0
    T1[0]=20#T0 #24 #T[0]
    Tav= T*0
    
    for i in range(1,len(v)):
        #T1[i]=T1[i-1]+(v[i]*np.abs(F[i]/1000)**alpha*(d1*T1[i-1]+d0)-T1[i-1]/(r0+r1*T1[i-1])+T[i]/(r0+r1*T1[i-1]))/(c1*T1[i-1]+c0)
        #T1[i]=T1[i-1]+(v[i]*np.abs(F[i]/1000)**alpha*(d1*T1[i-1]+d0)-T1[i-1]/(r0)+T[i]/(r0))/(c1*T1[i-1]+c0)
        #T1[i]=T1[i-1]+(v[i]*np.abs(F[i]/1000)**alpha*(d1*T1[i-1]+d0)-(T1[i-1]-T[i])/(r0+r1*T1[i-1]))
        T1[i]=T1[i-1]+(v[i]*np.abs(F[i]/1000)**(alpha)*(d1*T1[i-1]+d0)-(T1[i-1]-T[i])/(r0)-(np.abs(Tav[i-1]-T[i])**exp)/(a_1))
        if i<360:
            Tav[i]=np.mean(T1[0:i])
        else:
            Tav[i]=np.mean(T1[i-360:i])                   
        
        i+=1

    plt.plot(run_down.index, T1)
    #plt.plot(run_down.index, Tav)
    plt.plot(run_down.index, P, marker = '.')
    plt.ylim(0, 200)

    cost = np.sqrt(np.nansum((T1-P)**2))
    print(cost)
    
    return T1

# downsampling for now
res1 = plot_run(param, run_data_2['RR_in'].resample('10S').mean())
res2 = plot_run(param, run_data_2['RR_out'].resample('10S').mean())
res3 = plot_run(param, run_data_2['FR_in'].resample('10S').mean())
res4 = plot_run(param, run_data_2['FR_out'].resample('10S').mean())

res1 = plot_run(param, run_data['RR_in'].resample('10S').mean())
res2 = plot_run(param, run_data['RR_out'].resample('10S').mean())
res3 = plot_run(param, run_data['FR_in'].resample('10S').mean())
res4 = plot_run(param, run_data['FR_out'].resample('10S').mean())

"""
run = run_data_2['FR_in'].resample('10S').mean()
P = run['probe'].to_numpy()
diff = P-res1
t = run['probe'].index
fig = plt.figure(figsize =(4, 4)) 
plt.plot(t, diff, marker='.')
"""
plt.show()

"""
    1.087062487958899704e+00  # 1.437
    3.473807971019482466e-03
    -1.694147781945992833e-05
    4.552153537510104684e+02
    6.534897187119651819e-03
    2.303280305177073117e+01
"""

206.17397470326932
213.29095215686894
111.3259998021527
56.1990964675465
219.20330812736043
177.35511547133302
154.8258110564418
70.48233278778444


'\n    1.087062487958899704e+00  # 1.437\n    3.473807971019482466e-03\n    -1.694147781945992833e-05\n    4.552153537510104684e+02\n    6.534897187119651819e-03\n    2.303280305177073117e+01\n'